In [ ]:
#本番

In [1]:
# ライブラリimport
import argparse
import logging
import os

import pandas as pd
from pandas import DataFrame
import pandas.tseries.offsets as offsets
import numpy as np
from datetime import date, timedelta, time, datetime
from tabulate import tabulate

from itertools import product

from ailab_tools.smn import ImpalaResource
from ailab_tools.utils import GMail, GSheet

from collections import OrderedDict

import zipfile
import glob

import gc

from ailab_tools.magics import slack_notify

In [2]:
from pathlib import Path
from zipfile import ZipFile

In [3]:
data_dir = Path("/home/yusuke_akada/notebook/adhoc")

In [4]:
original_column_names = ['ip_address', 'office_id',
                         'biz_category_lv1_code1', 'biz_category_lv1_name1',
                         'biz_category_lv1_code2', 'biz_category_lv1_name2',
                         'biz_category_lv1_code3', 'biz_category_lv1_name3',
                         'biz_category_lv2_code1', 'biz_category_lv2_name1',
                         'biz_category_lv2_code2', 'biz_category_lv2_name2',
                         'biz_category_lv2_code3', 'biz_category_lv2_name3',
                         'biz_category_lv3_code1', 'biz_category_lv3_name1',
                         'biz_category_lv3_code2', 'biz_category_lv3_name2',
                         'biz_category_lv3_code3', 'biz_category_lv3_name3',
                         'biz_category_lv4_code1', 'biz_category_lv4_name1',
                         'biz_category_lv4_code2', 'biz_category_lv4_name2',
                         'biz_category_lv4_code3', 'biz_category_lv4_name3',
                         'prefecture_code', 'municipality_code', 'prefecture_name', 'municipality_name',
                         'listed_code', 'listed_name', 'capital', 'capital_range',
                         'employee_range', 'sales_range', 'sales_profit_range', 'meta_description',
                         'meta_keyword','isp_flag','duplication_flag','company_name','postal_code',
                         'address','phone_number','URL','domain','corporate_no','employee_range_r','sales_range_r']

In [5]:
len(original_column_names)

50

In [6]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 50)

zipを解凍して.txtファイルを開き、文字コードを変換する。

In [7]:
import csv
import io

In [33]:
for f in data_dir.iterdir():  # Dataのあるディレクトリ内のファイルを列挙
    if ".zip" == f.suffix:  # zipファイルだけを抽出
        with ZipFile(f) as zip_file:  # zipファイルを開く
            for f_name in zip_file.namelist():  # zipファイル内にあるファイルを列挙
                if ".TXT" == f_name[-4:]:  # txtファイルを開く
                    print(f"Opening and compressing {f_name} ...")
                    # 該当ファイルを解凍して、出力先のファイルを作る
                    with zip_file.open(f_name, mode="r") as data_source, open(data_dir / f"{f_name[:4]}_compressed.txt", "w") as output:
                        csv_file = csv.reader(io.TextIOWrapper(data_source, encoding="cp932"),
                                              delimiter=",", quotechar='"')  # csvとして展開する
                        for row in csv_file:  # 1行ごと見る
                            # utf-8に文字コードを変換
                            line = "\t".join(row)
                            decoded_line = line.encode("utf-8").decode("utf-8")
                            # ダブルクォーテーションを除いてタブ区切りにしてファイルに出力
                            output.write(decoded_line+ "\n")

Opening and compressing IP08.TXT ...
Opening and compressing IP13.TXT ...
Opening and compressing IP05.TXT ...
Opening and compressing IP17.TXT ...
Opening and compressing IP02.TXT ...
Opening and compressing IP16.TXT ...
Opening and compressing IP06.TXT ...
Opening and compressing IP01.TXT ...


In [8]:
import numpy as np

In [1]:
def concat_multi_cols(row):
    # どのカラムから何のカラムを生成したいかを決める
    col_recipe = [
        ("biz_category_lv1_code", ["biz_category_lv1_code1", "biz_category_lv1_code2", "biz_category_lv1_code3"]),
        ("biz_category_lv1_name", ["biz_category_lv1_name1", "biz_category_lv1_name2", "biz_category_lv1_name3"]),
        ("biz_category_lv2_code", ["biz_category_lv2_code1", "biz_category_lv2_code2", "biz_category_lv2_code3"]),
        ("biz_category_lv2_name", ["biz_category_lv2_name1", "biz_category_lv2_name2", "biz_category_lv2_name3"]),
        ("biz_category_lv3_code", ["biz_category_lv3_code1", "biz_category_lv3_code2", "biz_category_lv3_code3"]),
        ("biz_category_lv3_name", ["biz_category_lv3_name1", "biz_category_lv3_name2", "biz_category_lv3_name3"]),
        ("biz_category_lv4_code", ["biz_category_lv4_code1", "biz_category_lv4_code2", "biz_category_lv4_code3"]),
        ("biz_category_lv4_name", ["biz_category_lv4_name1", "biz_category_lv4_name2", "biz_category_lv4_name3"])
    ]
    
    # レシピ1つ1つからカラムを作る
    result_cols = {}
    for result_name, col_names in col_recipe:
        # コードのリストを作る
        codes = [row[col_names[0]], row[col_names[1]], row[col_names[2]]]

        # 重複を除外して、要素がstrかnanではないかをチェックして、良ければstr型に変換してリストにする.
        codes = [str(c) for c in set(codes) if isinstance(c, str) or not np.isnan(c)]
        
        # semmi-colon区切りの文字列にする
        code_str = ";".join(codes)
    
        # 結果を保存
        result_cols[result_name] = code_str
        
    return pd.Series(result_cols)

In [10]:
uid_table = "dm.landscape_ip_company"   # 最終的な対応table
temp_table = "dm_tmp.ya_tmp_landscape"    # 分割処理用のtemporary table 実行後dropされる

with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                    request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:

    #まず空tableを作る
    query = (
        f"""
        create table {uid_table}
            (
            ip_address string,
            office_id string,
            prefecture_code string,
            municipality_code string,
            prefecture_name string,
            municipality_name string,
            listed_code string,
            listed_name string,
            capital float,
            capital_range string,
            employee_range string,
            sales_range string,
            sales_profit_range string,
            meta_description string,
            meta_keyword string,
            isp_flag string,
            duplication_flag string,
            company_name string,
            postal_code string,
            address string,
            phone_number string, 
            URL string,
            domain string,
            corporate_no string,
            employee_range_r string,
            sales_range_r string,
            biz_category_lv1_code string,
            biz_category_lv1_name string,
            biz_category_lv2_code string,
            biz_category_lv2_name string,
            biz_category_lv3_code string,
            biz_category_lv3_name string,
            biz_category_lv4_code string,
            biz_category_lv4_name string
            )
        
        partitioned by 
            (
            block int
            )
        
        stored as parquet
        """
    )
    
    ir.execute_query_no_ret(query) 

In [11]:
%%slack_notify @yusuke_akada
# テキストファイル単体ごとに読込、DataFrame化
for i in range(1,22):
    if i < 10:
        df = pd.read_csv(data_dir / f"IP0{i}_compressed.txt", sep="\t", names=original_column_names,
                dtype={'biz_category_lv1_code1': str,
                       'biz_category_lv1_code2': str,
                       'biz_category_lv1_code3': str,
                       'biz_category_lv2_code1': str,
                       'biz_category_lv2_code2': str,
                       'biz_category_lv2_code3': str,
                       'biz_category_lv3_code1': str,
                       'biz_category_lv3_code2': str,
                       'biz_category_lv3_code3': str,
                       'biz_category_lv4_code1': str,
                       'biz_category_lv4_code2': str,
                       'biz_category_lv4_code3': str,
                       'prefecture_code': str, 
                       'municipality_code': str,
                       'listed_code': str,
                       'capital_range': str,
                       'employee_range': str,
                       'sales_range': str,
                       'sales_profit_range': str,
                       'meta_description': str,
                       'isp_flag': str,
                       'duplication_flag': str,
                       'employee_range_r': str,
                       'sales_range_r': str})
        df_convert = df.apply(concat_multi_cols, axis=1)
        df_merge = pd.concat([df, df_convert], axis=1)
        df_merge = df_merge.drop(['biz_category_lv1_code1','biz_category_lv1_name1','biz_category_lv1_code2','biz_category_lv1_name2',
                                  'biz_category_lv1_code3','biz_category_lv1_name3','biz_category_lv2_code1','biz_category_lv2_name1',
                                  'biz_category_lv2_code2','biz_category_lv2_name2','biz_category_lv2_code3','biz_category_lv2_name3',
                                  'biz_category_lv3_code1','biz_category_lv3_name1','biz_category_lv3_code2','biz_category_lv3_name2',
                                  'biz_category_lv3_code3','biz_category_lv3_name3','biz_category_lv4_code1','biz_category_lv4_name1',
                                  'biz_category_lv4_code2','biz_category_lv4_name2','biz_category_lv4_code3','biz_category_lv4_name3'],axis=1)
    else:
        df = pd.read_csv(data_dir / f"IP{i}_compressed.txt", sep="\t", names=original_column_names,
                dtype={'biz_category_lv1_code1': str,
                       'biz_category_lv1_code2': str,
                       'biz_category_lv1_code3': str,
                       'biz_category_lv2_code1': str,
                       'biz_category_lv2_code2': str,
                       'biz_category_lv2_code3': str,
                       'biz_category_lv3_code1': str,
                       'biz_category_lv3_code2': str,
                       'biz_category_lv3_code3': str,
                       'biz_category_lv4_code1': str,
                       'biz_category_lv4_code2': str,
                       'biz_category_lv4_code3': str,
                       'prefecture_code': str, 
                       'municipality_code': str,
                       'listed_code': str,
                       'capital_range': str,
                       'employee_range': str,
                       'sales_range': str,
                       'sales_profit_range': str,
                       'meta_description': str,
                       'isp_flag': str,
                       'duplication_flag': str,
                       'employee_range_r': str,
                       'sales_range_r': str})
        df_convert = df.apply(concat_multi_cols, axis=1)
        df_merge = pd.concat([df, df_convert], axis=1)
        df_merge = df_merge.drop(['biz_category_lv1_code1','biz_category_lv1_name1','biz_category_lv1_code2','biz_category_lv1_name2',
                                  'biz_category_lv1_code3','biz_category_lv1_name3','biz_category_lv2_code1','biz_category_lv2_name1',
                                  'biz_category_lv2_code2','biz_category_lv2_name2','biz_category_lv2_code3','biz_category_lv2_name3',
                                  'biz_category_lv3_code1','biz_category_lv3_name1','biz_category_lv3_code2','biz_category_lv3_name2',
                                  'biz_category_lv3_code3','biz_category_lv3_name3','biz_category_lv4_code1','biz_category_lv4_name1',
                                  'biz_category_lv4_code2','biz_category_lv4_name2','biz_category_lv4_code3','biz_category_lv4_name3'],axis=1)
        
    with ImpalaResource(hosts=["172.16.60.109"], port=21050, user='vmspool',
                        request_pool='vmspool', httpfs_host='hc3manager02.hc3pro', httpfs_port=14000) as ir:
        # ImpalaにUP
        ir.upload_dataframe(temp_table, df_merge, ['string','string','string','string','string','string','string','string','float','string',
                                                   'string','string','string','string','string','string','string','string','string','string',
                                                   'string','string','string','string','string','string','string','string','string','string',
                                                   'string','string','string','string'])
        # 親テーブルにInsert
        query = (
            f"""
            insert overwrite table {uid_table}
            (
            ip_address,
            office_id,
            prefecture_code,
            municipality_code,
            prefecture_name,
            municipality_name,
            listed_code,
            listed_name,
            capital,
            capital_range,
            employee_range,
            sales_range,
            sales_profit_range,
            meta_description,
            meta_keyword,
            isp_flag,
            duplication_flag,
            company_name,
            postal_code,
            address,
            phone_number, 
            URL,
            domain,
            corporate_no,
            employee_range_r,
            sales_range_r,
            biz_category_lv1_code,
            biz_category_lv1_name,
            biz_category_lv2_code,
            biz_category_lv2_name,
            biz_category_lv3_code,
            biz_category_lv3_name,
            biz_category_lv4_code,
            biz_category_lv4_name
            )
            partition
            (
            block = {i}
            )
            select
            ip_address,
            office_id,
            prefecture_code,
            municipality_code,
            prefecture_name,
            municipality_name,
            listed_code,
            listed_name,
            capital,
            capital_range,
            employee_range,
            sales_range,
            sales_profit_range,
            meta_description,
            meta_keyword,
            isp_flag,
            duplication_flag,
            company_name,
            postal_code,
            address,
            phone_number, 
            URL,
            domain,
            corporate_no,
            employee_range_r,
            sales_range_r,
            biz_category_lv1_code,
            biz_category_lv1_name,
            biz_category_lv2_code,
            biz_category_lv2_name,
            biz_category_lv3_code,
            biz_category_lv3_name,
            biz_category_lv4_code,
            biz_category_lv4_name
            from {temp_table}
            """
        )
        ir.execute_query_no_ret(query) 
        # tempテーブルdrop
        ir.drop_tables([temp_table])

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (1,5,7,11,13,17,19,23,25,38,45,46) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
INFO:ailab_tools.smn.impala_client:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.109']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.109
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : vmspool
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'vmspool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /opt/anaconda3/lib/python3.7/site-packages/ailab_tools/config/ailab_tools_config.py
I